Изменения с прошлого раза:
- добавлены пропущенные участники
- добавлены данные об участниках
- есть таймпоинт т1
- добавлена метрика pause_count
- данные приведены в нужный формат для LME моделей и выгружены в csv формат

In [2]:
path = "/Users/alinazabolotskaya/Desktop/Центр Языка и Мозга/дискурс проверенные/"
example = "/Users/alinazabolotskaya/Desktop/Центр Языка и Мозга/дискурс проверенные/мария 8 июль/ксения 2 все т3/JK6441.cha"
excel_path = "/Users/alinazabolotskaya/Desktop/Центр Языка и Мозга/table20220121.xlsx"
import os
import pylangacq as pla
import pandas as pd

In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alinazabolotskaya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 0) Проверка всех файлов:
- какие надо допроверить (в названии оставляла пометку "check on mac")
- маркировка [% bad quality] в тех, где аудио повреждено/невозможно улучшить до нормального качества
- повторяющиеся файлы с маркировкой "было" занести в отдельный список

In [4]:
listing = []
repeated = []
for root, dirs, files in os.walk(path):
    for file in files:
        pathfile = os.path.join(root, file)
        if file.endswith('было.cha'):
            repeated.append((pathfile, file.strip(' было.cha')))
        elif file.endswith('.cha'):
            listing.append((pathfile, file.strip('.cha')))
            #print(file)
        #sx = pla.read_chat(pathfile)
        #with open(pathfile, 'r') as reader:
         #   f = reader.read()
          #  pauses = pauses(f)

In [5]:
ex = pla.read_chat(example)

Читаю все файлы, записываю в словарь.

In [6]:
def file_to_dict(textfile):
    dic = {}
    headers = []
    inv = []
    par = []
    cur = 0
    cnt = 0
    for line in textfile.split('\n'):
        if line.startswith('@'):
            headers.append(line)
            if line.startswith('@Media'):
                name = line.strip('@Media:\t').strip(', audio')
        elif line.startswith('*PAR'):
            cur = 'par'
            cnt += 1
            par.append({cnt: line})
        elif line.startswith('*INV'):
            cnt += 1
            cur = 'inv'
            inv.append({cnt: line})
        elif line.startswith('*') == False:
            if cur == 'par':
                ku = par[-1][cnt] + ' ' + line
                par = par[:-1]
                par.append({cnt: ku})
            if cur == 'inv':
                ku = inv[-1][cnt] + ' ' + line
                inv = inv[:-1]
                inv.append({cnt: ku})
    par_table = []
    for p in par:
        prep = list(p.values())[0].split('\x15', 1)
        p_dic = {'n_utt': list(p.keys())[0], 
                'text_utt': prep[0].strip('*PAR:\t'),
                'time_utt': prep[-1].strip('\x15')}
        par_table.append(p_dic)
    try:
        point = name.split('_')[1]
    except IndexError:
        point = "T1"
    dic.update({'name': name.split('_')[0], 'headers': headers, 'par': par_table, 'inv': inv, 'point': point})
    return dic

In [7]:
all_files = []
for file in listing:
    filename = file[1]
    pathfile = file[0]
    with open(pathfile, 'r') as reader:
        textfile = reader.read()
        
        all_files.append({filename: file_to_dict(textfile)})

In [8]:
all_files[33]

{'OO6669': {'name': 'OO6669',
  'headers': ['@UTF8',
   '@Window:\t0_0_0_0_489_1_840_0_840_0',
   '@Begin',
   '@Languages:\trus',
   '@Participants:\tINV Investigator, PAR Participant',
   '@ID:\trus|change_corpus_later|INV|||||Investigator|||',
   '@ID:\trus|change_corpus_later|PAR|||||Participant|||',
   '@Media:\tOO6669, audio',
   '@End'],
  'par': [{'n_utt': 2,
    'text_utt': 'встреча нового года (0.296) &=mhm с друзьями. ',
    'time_utt': '3338_5805'},
   {'n_utt': 3,
    'text_utt': '&=mhm ежегодно (0.325) мы собираемся на встречу нового года. ',
    'time_utt': '6500_10424'},
   {'n_utt': 4,
    'text_utt': 'много друзей которые живут недалеко <от> [//] <там> [//] (0.240) от \tменя. ',
    'time_utt': '10877_14656'},
   {'n_utt': 5,
    'text_utt': 'сначала я встречаю новый год со своей семьей а потом прихожу к \tдрузьям. ',
    'time_utt': '15153_18624'},
   {'n_utt': 6,
    'text_utt': 'традиционная встреча нового года с родными (1.023) &=mhm бывает не \tтакой веселой (0.4

In [9]:
dic = file_to_dict(textfile)

### 1) Создание датафрейма с паузами

__1.1. Определяем тип паузы__
- Паузы, рядом с которыми есть пометка &=er, &-это_самое и т. д. считаются заполненными.
- Паузы, рядом с которыми нет такой пометки - пустые.

Для поиска определенных паттернов текста (в данном случае число в скобках и пометка "&=") используется модуль regex.
В рассмотрение берем только участника *PAR.

__1.2 Заносим в словарь тип пауз, их длину, контекст, номер реплики и говорящего.__

In [10]:
name_correct_mapper = { ## Исправляем неправильные имена участников в данных
    'B6780': 'MB6780',
'GZ7966': 'GL7966',
'IY26492': 'IY2649',
'JU5933': 'JY5933',
'KF9614': 'XF9614',
'N3454': 'MN3454',
'OO6669': 'OO6659',
'R3523': 'MR3523',
'V9573': 'MV9573',
'Z0085': 'MZ0085',
'ZH6469': 'ZM6469'
}
def name_correct(name):
    if name in name_correct_mapper.keys():
        return name_correct_mapper[name]
    else:
        return(name)

In [11]:
def get_pauses(dic):
    pause_dic = []
    cnt = 0
    point = list(dic.values())[0]['point']
    filename = list(dic.values())[0]['name']
    for f in list(dic.values())[0]['par']:
        utt = f['n_utt']
        text_utt = f['text_utt']
        text_utt = re.sub(r'&-', '&=', text_utt, re.DOTALL) # привожу к одному формату все пометы-указания на заполненные паузы
        if utt - cnt == 1 and utt != 1:
            betw_pause = int(f['time_utt'].split('_')[0]) - int(end_prev)
            if text_utt.startswith('&='):
                betw_pause_type = 'filled'
            else:
                betw_pause_type = 'empty'
            pause_dic.append({'participant': name_correct(filename), 'point': point, 'pause_type': betw_pause_type, 'pause_place': 'between_utt', 'pause_length': betw_pause * 0.001, 'context': text_prev + text_utt, 'utt_n': utt - 0.5})
        rawfilled = re.findall(r'\(\d+\.\d+\) &=\w+|&=\w+ \(\d+\.\d+\)', text_utt, re.DOTALL) # нахожу все вхождения заполненных пауз
        cnt = utt
        end_prev = f['time_utt'].split('_')[1]
        text_prev = text_utt
        for pause in rawfilled:
            pause = re.findall(r'\d+\.\d+', pause, re.DOTALL)[0]
            pause_dic.append({'participant': name_correct(filename), 'point': point, 'pause_type': 'filled', 'pause_place': 'in_utt', 'pause_length': float(pause), 'context': text_utt, 'utt_n': utt})
        all_pauses = re.findall(r'\(\d+\.\d+\)', text_utt, re.DOTALL)
        for pause in all_pauses:
            pause = float(pause.strip(')').strip('('))
            if pause not in rawfilled:
                pause_dic.append({'participant': name_correct(filename), 'point': point, 'pause_type': 'empty', 'pause_place': 'in_utt', 'pause_length': float(pause), 'context': text_utt, 'utt_n': utt})

    return pause_dic

In [12]:
all_pause_dic = []
cnt = 0
for file in all_files:
    all_pause_dic.extend(get_pauses(file))

__1.3 Создаем датафрейм__

In [13]:
pauses = pd.DataFrame(all_pause_dic)
pauses

,participant,point,pause_type,pause_place,pause_length,context,utt_n
0,JX9879,T1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0
1,JX9879,T1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0
2,JX9879,T1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5
3,JX9879,T1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5
4,JX9879,T1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0
...,...,...,...,...,...,...,...
2825,JY5933,T1,empty,between_utt,0.887,своей внучке на (0.251) шестнадцать лет (0.405...,3.5
2826,JY5933,T1,empty,in_utt,0.267,в результате конечно сделала (0.267) очень кра...,4.0
2827,JY5933,T1,empty,between_utt,0.635,в результате конечно сделала (0.267) очень кра...,4.5
2828,JY5933,T1,filled,between_utt,0.173,со свечками. &=ну получился очень красивый и е...,5.5


__1.4 Чистка датафрейма__

Убираем все паузы длиной менее 0.200, если такие есть. 
(Этот шаг, возможно, можно еще доработать, надо придумать, как.)

In [14]:
pauses.shape

(2830, 7)

In [15]:
pauses = pauses[pauses.pause_length >= 0.2]
pauses.shape

(2585, 7)

### 2) Подсчет метрик.

In [16]:
pauses.groupby('participant').mean()['pause_length'] # Средняя длина паузы по участнику

participant
BB5946    2.378722
BJ2008    1.066375
BU9305    0.562000
BZ3684    1.313186
CE9478    0.999500
            ...   
ZH6769    0.942350
ZK9063    0.730450
ZM6469    2.664500
ZU5403    1.011106
ZV8646    1.027083
Name: pause_length, Length: 87, dtype: float64

In [17]:
pauses.groupby(['participant', 'pause_type']).mean()['pause_length'] # Средняя длина пауз разных типов по участнику

participant  pause_type
BB5946       empty         1.924583
             filled        3.287000
BJ2008       empty         0.997571
             filled        1.548000
BU9305       empty         0.562000
                             ...   
ZM6469       filled        5.357000
ZU5403       empty         0.901077
             filled        1.341192
ZV8646       empty         0.940294
             filled        1.237857
Name: pause_length, Length: 155, dtype: float64

In [18]:
# Pause rate (length of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].sum() / pauses.groupby('participant')['utt_n'].mean()

participant
BB5946    6.881304
BJ2008    4.874857
BU9305    1.570807
BZ3684    5.477774
CE9478    1.923491
            ...   
ZH6769    1.948010
ZK9063    2.623389
ZM6469    3.856161
ZU5403    8.855158
ZV8646    2.777465
Length: 87, dtype: float64

In [19]:
# Pause per utterance (number of pauses of 1 speaker/number of utt
pauses.groupby('participant')['pause_length'].count()/ pauses.groupby('participant')['utt_n'].mean()

participant
BB5946    2.892857
BJ2008    4.571429
BU9305    2.795031
BZ3684    4.171360
CE9478    1.924453
            ...   
ZH6769    2.067183
ZK9063    3.591470
ZM6469    1.447236
ZU5403    8.757895
ZV8646    2.704225
Length: 87, dtype: float64

### Подготовка к подсчету корреляций в точке 1. 

In [20]:
df = pauses

In [21]:
df['point'] = df['point'].map(lambda x: int(x[1:]))

<ipython-input-21-3c58a4431e96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['point'].map(lambda x: int(x[1:]))


In [22]:
df1 = df.loc[df['point'] == 1]
df3 = df.loc[df['point'] == 3]

In [23]:
excel = pd.read_excel(excel_path, index_col=0)  

In [24]:
from datetime import date

def calculate_age(born):
    today = date.today()
    try:
        return today.year - int(born.split('.')[2]) - ((today.month, today.day) < (int(born.split('.')[1]), int(born.split('.')[0])))
    except AttributeError:
        return None

In [25]:
excel['Возраст'] = excel['Дата рождения'].apply(lambda x: calculate_age(x))

In [27]:
df1 = df1.merge(excel[['Unnamed: 143']], left_on='participant', right_on='Код пациента', how='left')
df1 = df1.merge(excel[['Unnamed: 102']], left_on='participant', right_on='Код пациента', how='left')
df3 = df3.merge(excel[['Unnamed: 303']], left_on='participant', right_on='Код пациента', how='left')
df3 = df3.merge(excel[['Unnamed: 363']], left_on='participant', right_on='Код пациента', how='left')
df1 = df1.rename(columns={'Unnamed: 143': 'moca', 'Unnamed: 102': 'mmse'})
df3 = df3.rename(columns={'Unnamed: 303': 'moca', 'Unnamed: 363': 'mmse'})

In [28]:
df0 = pd.concat([df1, df3], sort=False)

In [29]:
df0

,participant,point,pause_type,pause_place,pause_length,context,utt_n,moca,mmse
0,JX9879,1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27
1,JX9879,1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27
2,JX9879,1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,26,27
3,JX9879,1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,26,27
4,JX9879,1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,26,27
...,...,...,...,...,...,...,...,...,...
341,ZU5403,3,empty,in_utt,3.461,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN
342,ZU5403,3,empty,in_utt,1.616,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN
343,ZU5403,3,empty,in_utt,1.509,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN
344,ZU5403,3,empty,in_utt,1.486,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN


In [30]:
df0 = df0.merge(excel[['Возраст']], left_on='participant', right_on='Код пациента', how='left')

In [31]:
df0 = df0.rename(columns={'Возраст': 'age'})
df0

,participant,point,pause_type,pause_place,pause_length,context,utt_n,moca,mmse,age
0,JX9879,1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27,62.0
1,JX9879,1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27,62.0
2,JX9879,1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,26,27,62.0
3,JX9879,1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,26,27,62.0
4,JX9879,1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,26,27,62.0
...,...,...,...,...,...,...,...,...,...,...
2580,ZU5403,3,empty,in_utt,3.461,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2581,ZU5403,3,empty,in_utt,1.616,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2582,ZU5403,3,empty,in_utt,1.509,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2583,ZU5403,3,empty,in_utt,1.486,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0


In [32]:
df0

,participant,point,pause_type,pause_place,pause_length,context,utt_n,moca,mmse,age
0,JX9879,1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27,62.0
1,JX9879,1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26,27,62.0
2,JX9879,1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,26,27,62.0
3,JX9879,1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,26,27,62.0
4,JX9879,1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,26,27,62.0
...,...,...,...,...,...,...,...,...,...,...
2580,ZU5403,3,empty,in_utt,3.461,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2581,ZU5403,3,empty,in_utt,1.616,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2582,ZU5403,3,empty,in_utt,1.509,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0
2583,ZU5403,3,empty,in_utt,1.486,&=er мы дышали свежим &=mhm (0.730) морским во...,22.0,NaN,NaN,58.0


In [33]:
df0 = df0[df0['moca'].notna()]
df0 = df0[df0['mmse'].notna()]

In [34]:
df0['mmse'] = df0['mmse'].apply(lambda x: float(x))
df0['moca'] = df0['moca'].apply(lambda x: float(x))

In [35]:
from scipy.stats import pearsonr

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

# Корреляции:

In [36]:
df0.groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы и mmse: -0.254062
#Коэффициент корреляции Пирсона между средней длиной паузы и moca: -0.176292
#Коэффициент корреляции Пирсона между средней длиной паузы и возрастом: 0.148773

,point,pause_length,utt_n,moca,mmse,age
point,1.000000,-0.035378,-0.009807,0.084083,-0.945388,0.110069
pause_length,-0.035378,1.000000,0.010624,-0.178612,-0.043213,0.096589
utt_n,-0.009807,0.010624,1.000000,-0.142763,-0.027720,0.094003
moca,0.084083,-0.178612,-0.142763,1.000000,0.121600,-0.427956
mmse,-0.945388,-0.043213,-0.027720,0.121600,1.000000,-0.181162
age,0.110069,0.096589,0.094003,-0.427956,-0.181162,1.000000


In [37]:
calculate_pvalues(df0.groupby('participant').mean())
#P-value средней длины паузы и mmse: 0.0395
##P-value средней длины паузы и moca: 0.1568
#P-value средней длины паузы и возраста: 0.8225

,point,pause_length,utt_n,moca,mmse,age
point,0,0.7554,0.9312,0.4584,0,0.3311
pause_length,0.7554,0,0.9255,0.1129,0.7035,0.394
utt_n,0.9312,0.9255,0,0.2065,0.8072,0.4069
moca,0.4584,0.1129,0.2065,0,0.2826,0.0001
mmse,0,0.7035,0.8072,0.2826,0,0.1078
age,0.3311,0.394,0.4069,0.0001,0.1078,0


In [39]:
df0.loc[df0['pause_type'] == 'empty'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и mmse: -0.270230
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и moca: -0.189789
#Коэффициент корреляции Пирсона между средней длиной незаполненной паузы и возрастом: 0.166481

,point,pause_length,utt_n,moca,mmse,age
point,1.000000,-0.019233,-0.012626,0.084594,-0.945677,0.106572
pause_length,-0.019233,1.000000,0.007372,-0.179680,-0.067402,0.088882
utt_n,-0.012626,0.007372,1.000000,-0.138440,-0.021915,0.095692
moca,0.084594,-0.179680,-0.138440,1.000000,0.120722,-0.426976
mmse,-0.945677,-0.067402,-0.021915,0.120722,1.000000,-0.177683
age,0.106572,0.088882,0.095692,-0.426976,-0.177683,1.000000


In [41]:
calculate_pvalues(df0.loc[df0['pause_type'] == 'empty'].groupby('participant').mean())
#P-value средней длины незаполненной паузы и mmse: 0.0282
##P-value средней длины незаполненной паузы и moca: 0.1269
#P-value средней длины незаполненной паузы и возраста: 0.6988

,point,pause_length,utt_n,moca,mmse,age
point,0,0.8655,0.9115,0.4556,0,0.3468
pause_length,0.8655,0,0.9483,0.1108,0.5525,0.433
utt_n,0.9115,0.9483,0,0.2207,0.847,0.3985
moca,0.4556,0.1108,0.2207,0,0.2861,0.0001
mmse,0,0.5525,0.847,0.2861,0,0.1148
age,0.3468,0.433,0.3985,0.0001,0.1148,0


In [170]:
df0.loc[df0['pause_type'] == 'filled'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и mmse: -0.313677
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и moca: -0.179320
#Коэффициент корреляции Пирсона между средней длиной заполненной паузы и возрастом: -0.116277

,point,pause_length,utt_n,moca,mmse,age
point,1.000000,-0.100382,-0.012203,0.054692,-0.963529,0.191086
pause_length,-0.100382,1.000000,0.154874,-0.183416,0.033689,0.024174
utt_n,-0.012203,0.154874,1.000000,-0.218122,-0.037084,0.141101
moca,0.054692,-0.183416,-0.218122,1.000000,0.110386,-0.416459
mmse,-0.963529,0.033689,-0.037084,0.110386,1.000000,-0.241579
age,0.191086,0.024174,0.141101,-0.416459,-0.241579,1.000000


In [42]:
calculate_pvalues(df0.loc[df0['pause_type'] == 'filled'].groupby('participant').mean())
#P-value средней длины заполненной паузы и mmse: 0.0265
##P-value средней длины заполненной паузы и moca: 0.2127
#P-value средней длины заполненной паузы и возраста: 0.4213

,point,pause_length,utt_n,moca,mmse,age
point,0,0.3565,0.9168,0.6675,0,0.1407
pause_length,0.3565,0,0.2668,0.1246,0.7246,0.9092
utt_n,0.9168,0.2668,0,0.1176,0.7477,0.2776
moca,0.6675,0.1246,0.1176,0,0.392,0.0007
mmse,0,0.7246,0.7477,0.392,0,0.0581
age,0.1407,0.9092,0.2776,0.0007,0.0581,0


In [43]:
df0.loc[df0['pause_place'] == 'in_utt'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и mmse: -0.070749
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и moca: -0.089933
#Коэффициент корреляции Пирсона между средней длиной паузы внутри ЭДЕ и возрастом: 0.024157

,point,pause_length,utt_n,moca,mmse,age
point,1.000000,-0.025177,-0.020265,0.080021,-0.944705,0.121208
pause_length,-0.025177,1.000000,-0.016220,-0.099991,0.000328,-0.037215
utt_n,-0.020265,-0.016220,1.000000,-0.117208,-0.012807,0.075318
moca,0.080021,-0.099991,-0.117208,1.000000,0.126159,-0.430930
mmse,-0.944705,0.000328,-0.012807,0.126159,1.000000,-0.192094
age,0.121208,-0.037215,0.075318,-0.430930,-0.192094,1.000000


In [44]:
calculate_pvalues(df0.loc[df0['pause_place'] == 'in_utt'].groupby('participant').mean())
#P-value средней длины паузы внутри ЭДЕ и mmse: 0.5724
#P-value средней длины паузы внутри ЭДЕ и moca: 0.4727
#P-value средней длины паузы внутри ЭДЕ и возраста: 0.2819

,point,pause_length,utt_n,moca,mmse,age
point,0,0.8246,0.8584,0.4804,0,0.2842
pause_length,0.8246,0,0.8864,0.3775,0.9977,0.7431
utt_n,0.8584,0.8864,0,0.3005,0.9102,0.5067
moca,0.4804,0.3775,0.3005,0,0.2648,0.0001
mmse,0,0.9977,0.9102,0.2648,0,0.0878
age,0.2842,0.7431,0.5067,0.0001,0.0878,0


In [45]:
df0.loc[df0['pause_place'] == 'between_utt'].groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и moca: -0.167532
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и mmse: -0.275518
#Коэффициент корреляции Пирсона между средней длиной паузы между ЭДЕ и возрастом: 0.143556

,point,pause_length,utt_n,moca,mmse,age
point,1.000000,-0.059916,0.000157,0.080951,-0.947858,0.096692
pause_length,-0.059916,1.000000,-0.010503,-0.173020,-0.029550,0.088052
utt_n,0.000157,-0.010503,1.000000,-0.211251,-0.057106,0.151734
moca,0.080951,-0.173020,-0.211251,1.000000,0.125001,-0.417382
mmse,-0.947858,-0.029550,-0.057106,0.125001,1.000000,-0.174496
age,0.096692,0.088052,0.151734,-0.417382,-0.174496,1.000000


In [46]:
calculate_pvalues(df0.loc[df0['pause_place'] == 'between_utt'].groupby('participant').mean())
#P-value средней длины паузы между ЭДЕ и mmse: 0.0263
#P-value средней длины паузы между ЭДЕ и moca: 0.1822
#P-value средней длины паузы между ЭДЕ и возраста: 0.713

,point,pause_length,utt_n,moca,mmse,age
point,0,0.5999,0.9989,0.4782,0,0.3966
pause_length,0.5999,0,0.9268,0.1273,0.796,0.4403
utt_n,0.9989,0.9268,0,0.0616,0.6172,0.1819
moca,0.4782,0.1273,0.0616,0,0.2724,0.0001
mmse,0,0.796,0.6172,0.2724,0,0.124
age,0.3966,0.4403,0.1819,0.0001,0.124,0


In [47]:
pause_count = df0.groupby('participant')['pause_length'].count() / df0.groupby('participant')['utt_n'].mean()
#Pause rate (total number of pauses of 1 speaker/total number of utt of 1 spker)

In [48]:
df0 = df0.merge(pause_count.rename('pause_count'), left_on='participant', right_on='participant', how='left')

In [49]:
df0.groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между pause_count и moca: 0.126150
#Коэффициент корреляции Пирсона между pause_count и mmse: 0.125887
#Коэффициент корреляции Пирсона между pause_count и возрастом: -0.193524

,point,pause_length,utt_n,moca,mmse,age,pause_count
point,1.000000,-0.035378,-0.009807,0.084083,-0.945388,0.110069,0.002968
pause_length,-0.035378,1.000000,0.010624,-0.178612,-0.043213,0.096589,-0.093405
utt_n,-0.009807,0.010624,1.000000,-0.142763,-0.027720,0.094003,-0.222593
moca,0.084083,-0.178612,-0.142763,1.000000,0.121600,-0.427956,0.145120
mmse,-0.945388,-0.043213,-0.027720,0.121600,1.000000,-0.181162,0.037595
age,0.110069,0.096589,0.094003,-0.427956,-0.181162,1.000000,-0.196952
pause_count,0.002968,-0.093405,-0.222593,0.145120,0.037595,-0.196952,1.000000


In [50]:
pause_rate = df0.groupby('participant')['pause_length'].sum() / df0.groupby('participant')['utt_n'].mean()
#Pause rate (length of pauses of 1 speaker/number of utt

In [51]:
df0 = df0.merge(pause_rate.rename('pause_rate'), left_on='participant', right_on='participant', how='left')

In [52]:
df0.groupby('participant').mean().corr(method='pearson')
#Коэффициент корреляции Пирсона между pause_rate и moca: 0.048150
#Коэффициент корреляции Пирсона между pause_rate и mmse: 0.021145
#Коэффициент корреляции Пирсона между pause_rate и возрастом: -0.161589

,point,pause_length,utt_n,moca,mmse,age,pause_count,pause_rate
point,1.000000,-0.035378,-0.009807,0.084083,-0.945388,0.110069,0.002968,-0.036358
pause_length,-0.035378,1.000000,0.010624,-0.178612,-0.043213,0.096589,-0.093405,0.456264
utt_n,-0.009807,0.010624,1.000000,-0.142763,-0.027720,0.094003,-0.222593,-0.173739
moca,0.084083,-0.178612,-0.142763,1.000000,0.121600,-0.427956,0.145120,0.063588
mmse,-0.945388,-0.043213,-0.027720,0.121600,1.000000,-0.181162,0.037595,0.041302
age,0.110069,0.096589,0.094003,-0.427956,-0.181162,1.000000,-0.196952,-0.166396
pause_count,0.002968,-0.093405,-0.222593,0.145120,0.037595,-0.196952,1.000000,0.800255
pause_rate,-0.036358,0.456264,-0.173739,0.063588,0.041302,-0.166396,0.800255,1.000000


In [53]:
calculate_pvalues(df0.groupby('participant').mean())
#P-value pause_rate и mmse: 0.9417	
#P-value pause_rate и moca: 0.5512
#P-value pause_rate и возраста: 0.0981

,point,pause_length,utt_n,moca,mmse,age,pause_count,pause_rate
point,0,0.7554,0.9312,0.4584,0,0.3311,0.9792,0.7488
pause_length,0.7554,0,0.9255,0.1129,0.7035,0.394,0.4099,0
utt_n,0.9312,0.9255,0,0.2065,0.8072,0.4069,0.0472,0.1233
moca,0.4584,0.1129,0.2065,0,0.2826,0.0001,0.199,0.5752
mmse,0,0.7035,0.8072,0.2826,0,0.1078,0.7406,0.716
age,0.3311,0.394,0.4069,0.0001,0.1078,0,0.0799,0.1402
pause_count,0.9792,0.4099,0.0472,0.199,0.7406,0.0799,0,0
pause_rate,0.7488,0,0.1233,0.5752,0.716,0.1402,0,0


# Linear mixed-effect models
Сами модели мы будем делать в R, а сейчас подготовим датафрейм в формате csv.

In [54]:
df0

,participant,point,pause_type,pause_place,pause_length,context,utt_n,moca,mmse,age,pause_count,pause_rate
0,JX9879,1,empty,in_utt,0.279,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26.0,27.0,62.0,2.976744,3.647814
1,JX9879,1,empty,in_utt,1.330,хочу рассказать о поездке (0.279) на дачу (1.3...,1.0,26.0,27.0,62.0,2.976744,3.647814
2,JX9879,1,empty,between_utt,1.500,хочу рассказать о поездке (0.279) на дачу (1.3...,1.5,26.0,27.0,62.0,2.976744,3.647814
3,JX9879,1,filled,between_utt,2.560,погода стояла отвратительная. &=uh &+с громыха...,2.5,26.0,27.0,62.0,2.976744,3.647814
4,JX9879,1,empty,in_utt,1.185,&=uh &+с громыхал (1.185) гром.,3.0,26.0,27.0,62.0,2.976744,3.647814
...,...,...,...,...,...,...,...,...,...,...,...,...
2335,VQ9292,3,empty,between_utt,2.526,ну естественно радовалась (1.224) ходила такая...,12.5,29.0,1.0,69.0,2.161677,2.821671
2336,VQ9292,3,empty,between_utt,0.862,очень любила свои сапожки. не хотелось мне (1....,13.5,29.0,1.0,69.0,2.161677,2.821671
2337,VQ9292,3,empty,in_utt,1.469,не хотелось мне (1.469) <больно то> [?] +...,14.0,29.0,1.0,69.0,2.161677,2.821671
2338,VQ9292,3,empty,between_utt,1.670,не хотелось мне (1.469) <больно то> [?] +... я...,14.5,29.0,1.0,69.0,2.161677,2.821671


In [55]:
lme_df = df0.groupby(['participant','point']).mean()
lme_df = lme_df.rename(columns={'pause_length': 'mean_pause_length'})
lme_df = lme_df.drop(columns = 'utt_n')
lme_df = lme_df.reset_index()
lme_df

,participant,point,mean_pause_length,moca,mmse,age,pause_count,pause_rate
0,BB5946,1,2.378722,24.0,30.0,74.0,2.892857,6.881304
1,BJ2008,1,1.066375,29.0,27.0,74.0,4.571429,4.874857
2,BU9305,1,0.562000,25.0,27.0,63.0,2.795031,1.570807
3,BZ3684,1,1.313186,26.0,26.0,76.0,4.171360,5.477774
4,CE9478,1,0.999500,18.0,27.0,77.0,1.924453,1.923491
...,...,...,...,...,...,...,...,...
76,ZH6769,1,0.942350,27.0,27.0,68.0,2.067183,1.948010
77,ZK9063,1,0.837296,29.0,27.0,64.0,3.591470,2.623389
78,ZK9063,3,0.508538,22.0,1.0,64.0,3.591470,2.623389
79,ZM6469,1,2.664500,21.0,27.0,83.0,1.447236,3.856161


In [56]:
lme_df['mean_empty_pause_length'] = list(df.loc[df['pause_type'] == 'empty'].groupby('participant').mean()['pause_length'])
#lme_df['mean_filled_pause_length'] = list(df.loc[df['pause_type'] == 'filled'].groupby('participant').mean()['pause_length'])
#lme_df['mean_pause_length_between_utt'] = list(df.loc[df['pause_place'] == 'between_utt'].groupby('participant').mean()['pause_length'])
#lme_df['mean_pause_length_in_utt'] = list(df.loc[df['pause_place'] == 'in_utt'].groupby('participant').mean()['pause_length'])

ValueError: Length of values does not match length of index

In [57]:
mean_filled_pause_length = df.loc[df['pause_type'] == 'filled'].groupby('participant').mean()['pause_length'].reset_index().rename(columns={'pause_length': 'mean_filled_pause_length'})
mean_empty_pause_length = df.loc[df['pause_type'] == 'empty'].groupby('participant').mean()['pause_length'].reset_index().rename(columns={'pause_length': 'mean_empty_pause_length'})
mean_pause_length_between_utt = df.loc[df['pause_place'] == 'between_utt'].groupby('participant').mean()['pause_length'].reset_index().rename(columns={'pause_length': 'mean_pause_length_between_utt'})
mean_pause_length_in_utt = df.loc[df['pause_place'] == 'in_utt'].groupby('participant').mean()['pause_length'].reset_index().rename(columns={'pause_length': 'mean_pause_length_in_utt'})                                                                        

In [60]:
lme_df = lme_df.merge(mean_filled_pause_length)
lme_df = lme_df.merge(mean_empty_pause_length)
lme_df = lme_df.merge(mean_pause_length_between_utt)
lme_df = lme_df.merge(mean_pause_length_in_utt)
lme_df.tail()

,participant,point,mean_pause_length,moca,mmse,age,pause_count,pause_rate,mean_filled_pause_length,mean_empty_pause_length,mean_pause_length_between_utt,mean_pause_length_in_utt
59,ZH6769,1,0.942350,27.0,27.0,68.0,2.067183,1.948010,0.581000,0.961368,1.281889,0.664545
60,ZK9063,1,0.837296,29.0,27.0,64.0,3.591470,2.623389,1.031286,0.666636,0.996400,0.570880
61,ZK9063,3,0.508538,22.0,1.0,64.0,3.591470,2.623389,1.031286,0.666636,0.996400,0.570880
62,ZM6469,1,2.664500,21.0,27.0,83.0,1.447236,3.856161,5.357000,1.767000,3.209286,1.901800
63,ZV8646,1,1.027083,29.0,30.0,72.0,2.704225,2.777465,1.237857,0.940294,0.631333,1.159000


In [61]:
lme_df.to_csv('/Users/alinazabolotskaya/Desktop/Центр Языка и Мозга/mixed/preprocessed_for_lme.csv')